In [ ]:
# The following codes and networks are original work by Yasinda Samaranayake in 2023.

In [ ]:
import numpy as np
import keras
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

In [ ]:
np.random.seed(20)

In [ ]:
train_raw = loadmat('train_32x32.mat') #73257 training images
test_raw = loadmat('test_32x32.mat') #26032 testing images

In [ ]:
train_images = np.array(train_raw['X'])
test_images = np.array(test_raw['X'])

train_labels = train_raw['y']
test_labels = test_raw['y']

In [ ]:
print(train_images.shape)
print(test_images.shape)

In [ ]:
train_images = np.moveaxis(train_images, -1, 0)
test_images = np.moveaxis(test_images, -1, 0)

print(train_images.shape)
print(test_images.shape)

In [ ]:
new_images = []
new_labels = []
class_threshold = 50

one=0
two=0
three=0
four=0
five=0
six=0
seven=0
eight=0
nine=0
zero=0

for i in range(len(train_images)): #0-60000
    if train_labels[i][0]==10:
        if zero<class_threshold:
            new_images.append(train_images[i])
            new_labels.append(train_labels[i])
            zero+=1
    elif train_labels[i][0]==1:
        if one<class_threshold:
            new_images.append(train_images[i])
            new_labels.append(train_labels[i])
            one+=1
    elif train_labels[i][0]==2:
        if two<class_threshold:
            new_images.append(train_images[i])
            new_labels.append(train_labels[i])
            two+=1
    elif train_labels[i][0]==3:
        if three<class_threshold:
            new_images.append(train_images[i])
            new_labels.append(train_labels[i])
            three+=1
    elif train_labels[i][0]==4:
        if four<class_threshold:
            new_images.append(train_images[i])
            new_labels.append(train_labels[i])
            four+=1
    elif train_labels[i][0]==5:
        if five<class_threshold:
            new_images.append(train_images[i])
            new_labels.append(train_labels[i])
            five+=1
    elif train_labels[i][0]==6:
        if six<class_threshold:
            new_images.append(train_images[i])
            new_labels.append(train_labels[i])
            six+=1
    elif train_labels[i][0]==7:
        if seven<class_threshold:
            new_images.append(train_images[i])
            new_labels.append(train_labels[i])
            seven+=1
    elif train_labels[i][0]==8:
        if eight<class_threshold:
            new_images.append(train_images[i])
            new_labels.append(train_labels[i])
            eight+=1
    elif train_labels[i][0]==9:
        if nine<class_threshold:
            new_images.append(train_images[i])
            new_labels.append(train_labels[i])
            nine+=1

In [ ]:
train_images = np.array(new_images)
train_lbl = np.array(new_labels)

unique, counts = np.unique(train_lbl, return_counts=True)
dict(zip(unique, counts)) #show frequency of 500 training images

In [ ]:
train_images = train_images.astype('float64')
test_images = test_images.astype('float64')

In [ ]:
train_labels = train_lbl.astype('int64')
test_labels = test_labels.astype('int64')

In [ ]:
print('Min: {}, Max: {}'.format(train_images.min(), train_images.max()))

train_images /= 255.0
test_images /= 255.0

In [ ]:
lb = LabelBinarizer()
train_labels = lb.fit_transform(train_labels)
test_labels = lb.fit_transform(test_labels)

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels,
#                                                   test_size=0.15, random_state=22)

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/FYP/Saving weights

In [ ]:
import tensorflow
import numpy as np
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32,32,3)))
model.add(Conv2D(64, (3, 3),activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.50))
model.add(Conv2D(64, (3, 3),activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Dense(128,activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.50))
model.add(Flatten())
model.add(Dense(128,activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

cb = tensorflow.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    min_delta=0,
    patience=20,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False
)

weights_save = ModelCheckpoint('TUNED2SVHN.hdf5', save_best_only=True, monitor='val_loss', mode='min')


from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,
        featurewise_std_normalization=False,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range = 0.2,
        horizontal_flip=False)

datagen.fit(train_images)

history = model.fit(datagen.flow(train_images, train_labels, batch_size=16),
                    epochs=100,
                    verbose=2,
                    validation_data=(test_images, test_labels), callbacks=[cb, weights_save])